In [1]:
import pandas as pd
import requests
import pandas as pd
import time
from tqdm import tqdm
from bs4 import BeautifulSoup

In [2]:
# 업종별 지수 뭐 있는지랑 해당 사이트 내에서 어떤 코드로 이용되는지 수집.
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36'}
url = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'

mktsel = 1
data = {
    'locale': 'ko_KR',
    'mktsel': mktsel,
    'bld' : 'dbms/comm/finder/finder_equidx'   
    }
res = requests.post(url, data, headers = headers)
idf = pd.DataFrame(res.json()['block1'])
idf

,full_code,short_code,codeName,marketCode,marketName
0,5,300,KRX 300,KRX,KRX
1,5,600,KTOP 30,KRX,KRX
2,5,042,KRX 100,KRX,KRX
3,5,043,KRX 자동차,KRX,KRX
4,5,044,KRX 반도체,KRX,KRX
...,...,...,...,...,...
153,5,422,KRX 반도체 Top 15,KRX,테마
154,5,442,KRX FactSet 모빌리티 이노베이터 지수,KRX,테마
155,5,443,KRX FactSet 디지털 헬스케어 지수,KRX,테마
156,5,444,KRX FactSet 디지털 인프라 지수,KRX,테마


In [5]:
# 우선은 KOSPI 지수 데이터 먼저 획득.
idf = idf[idf['marketName'] == 'KOSPI']

# post방식으로 데이터 받아옴. 
# 모든 지수 10년치 데이터 수집(코스피 기준)
for index, row in idf.iterrows():
    print(row)
    index_name = row['codeName']
    indIdx2 = row['short_code']
    indIdx = row['full_code']
    
    data = {
        'bld':  'dbms/MDC/STAT/standard/MDCSTAT00301',
        'locale': 'ko_KR',
        'tboxindIdx_finder_equidx0_5':  index_name,
        'indIdx': indIdx,
        'indIdx2': indIdx2,
        'codeNmindIdx_finder_equidx0_5': index_name,
        'strtDd': '20150101',
        'endDd': '20230413',
        'share' : '2',
        'money': '3',
        'csvxls_isNo' : 'false'
    }

    res = requests.post(url, data, headers = headers)
    df = pd.DataFrame(res.json()['output'])
    df['index_name'] = index_name

    df.rename(columns = {'TRD_DD' : 'date',
                            'CLSPRC_IDX' : '종가',
                            'HGPRC_IDX' : '고가',
                            'LWPRC_IDX' : '저가',
                            'ACC_TRDVOL' : '거래량',
                            'ACC_TRDVAL' : '거래대금',
                            'MKTCAP' : '상장시가총액'}, inplace = True)
    
    i_name = index_name.replace('/','')
    df.to_csv(f'data\\{i_name}.csv', encoding = 'euc-kr')
    time.sleep(100/60)

full_code         1
short_code      001
codeName        코스피
marketCode      STK
marketName    KOSPI
Name: 28, dtype: object
full_code           1
short_code        028
codeName      코스피 200
marketCode        STK
marketName      KOSPI
Name: 29, dtype: object
full_code           1
short_code        034
codeName      코스피 100
marketCode        STK
marketName      KOSPI
Name: 30, dtype: object
full_code          1
short_code       035
codeName      코스피 50
marketCode       STK
marketName     KOSPI
Name: 31, dtype: object
full_code                1
short_code             167
codeName      코스피 200 중소형주
marketCode             STK
marketName           KOSPI
Name: 32, dtype: object
full_code                    1
short_code                 182
codeName      코스피 200 초대형제외 지수
marketCode                 STK
marketName               KOSPI
Name: 33, dtype: object
full_code                  1
short_code               244
codeName      코스피200제외 코스피지수
marketCode               STK
marketName             KO